# Wstęp do uczenia maszynowego - Projekt 1
## Aleksander Malinowski | Damian Skowroński
___
### Milestone 2
Będziemy robić modele na danych, na których wykonaliśmy preprocessing w kamieniu milowym 1. W preprocessing wykonaliśmy następujące zmiany:

*   dodatnie kolumn:
    *   _black_king_dst_ - odleglość czarnego króla od brzegu planszy
    *   _black2white_king_dst_ - odległość króli od siebie 
    *   _black2rook_dst_ - odległość czarnego króla od białej wieży
*   wykonanie label encoding dla zmiennych określających _file_
*   zamienienie wartości słownych w zmiennej celu _result_ na odpowiadające im liczby i wartości "draw" na "-1"

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

df = pd.read_csv("training_set.csv", index_col=0).reset_index(drop=True)
df.head()

,white_king_file,white_king_rank,white_rook_file,white_rook_rank,black_king_file,black_king_rank,result,black_king_dst,black2white_king_dst,black2rook_dst
0,2,2,3,3,8,3,13,0,6,5
1,1,1,6,3,5,3,-1,2,4,1
2,4,4,7,5,8,2,6,0,4,3
3,3,1,7,3,2,7,13,1,6,5
4,4,1,2,5,8,5,13,0,4,6


Podział na zbiór treningowy i walidacyjny

Podchodzimy do problemu przewidzenia rezultatu w następujący sposób:

1.  klasyfikacja, czy doszło do remisu "draw" (u nas wartość "-1"), czy nie (dowolna inna wartość)
2. regresja w celu przewidzenia ilości ruchów, dla wyników, które w pierwszym kroku zostały sklasyfikowane, że gra nie skończyła się remisem



### Klasyfikacja 
Ponieważ zbiór danych teoretycznie przedstawia optymalne ruchy, a przewaga białego nad czarnym jest duża, remis zachodzi zapewne w chwili kiedy czarny bije białą wieżę. Taka sytuacja zdaży się w momentcie, gdy czarny król jest w odległości od białej wieży równej "1", a biały król jest w odległości od białej wieży większej niż "1". W tym celu tworzę nową zmienną _white2rook_dst_, której nie braliśmy pod uwagę w preprocessingu (możliwe, że przyda się też potem w regresji).

In [2]:
def white2rookdistance(df):
     # dystans pomiędzy białym królem i wieżą
    return max(abs(df['white_king_file'] - df['white_rook_file']),abs(df['white_king_rank'] - df['white_rook_rank']))

Następnie ze otrzymanej nowej zmiennej _white2rook_dst_ i wcześniejszej _black2rook_dst_ mogę dowiedzieć się czy czarny zbije wieżę.

In [3]:
def w2r_rt(df): #white to rook distance + rook taken
    df['white2rook_dst'] = df.apply(lambda x: white2rookdistance(x), axis = 1)
    df["rook_taken"] = np.where(
        (df.black2rook_dst == 1) & (df.white2rook_dst != 1),
        0,1)

Sprawdzając korelację pomiędzy nową zmienna _rook_taken_, a result w wersji remis/nieremis otrzymujemy następujący wynik:

In [4]:
w2r_rt(df)
np.corrcoef(df.rook_taken,df.result.apply(lambda x: 0 if x == -1 else 1))

array([[1.        , 0.99858054],
       [0.99858054, 1.        ]])

Wychodzi, że korelacja jest bardzo duża.

In [5]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df.rook_taken,df.result.apply(lambda x: 0 if x == -1 else 1))

array([[ 1952,     0],
       [    5, 17682]], dtype=int64)

Okazuje się, nowa dokładnie opisuję sytuacje i popełniła tylko 5 błędów. Wobec tego myślę, że nie trzeba robić modelu do sklasyfikowania, a po prostu zdać się na nią. 

### 2. Regresja
Regresję wykonujemy w celu dowiedzenia się ile ruchów będzie potrzebne do zamatowania. Wykonywać ją będziemy na części obserwacji, które w poprzedniej "klasyfikacji" nie zwróciły remisu, czyli po prostu na tych, które mają "0" w nowej zmiennej _rook_taken_. 

Na początku jednak podzielimy aktualny zbiór.

In [6]:
df_sorted = df.sort_values("rook_taken").reset_index(drop=True)

In [7]:
df_sorted_dropped = df_sorted.drop(index=np.where(df_sorted.rook_taken == 0)[0])
df_sorted_dropped.value_counts('result')

result
 14    3187
 13    2936
 12    2518
 11    1998
 15    1516
 10    1389
 9     1198
 8     1003
 7      478
 6      414
 5      330
 16     273
 2      172
 4      139
 3       57
 1       55
 0       19
-1        5
dtype: int64

In [8]:
train_X, test_X, train_y, test_y = train_test_split(df_sorted_dropped.drop(columns = ["result"]), df_sorted_dropped.result, test_size = 0.3, random_state= 420, stratify=df_sorted_dropped.result)

In [31]:
#### .........................
from sklearn import svm
regr = svm.SVR()
regr.fit(train_X,train_y)

SVR()

In [35]:
y_pred = np.round(regr.predict(test_X)).astype(int)

In [44]:
y_pred

array([12,  7, 14, ..., 15, 13, 14])

In [47]:
regr.score(test_X,test_y)

0.8042820769243767

In [19]:
from sklearn import svm
svr_rbf = svm.SVR(kernel="rbf", C=30, gamma=0.1, epsilon=0.1)
svr_lin = svm.SVR(kernel="linear", C=10, gamma="auto")
svr_poly = svm.SVR(kernel="poly", C=1, gamma="auto", degree=2, epsilon=0.1, coef0=1)

In [20]:
svr_rbf.fit(train_X,train_y)
svr_rbf.score(test_X,test_y)

0.8880910275213326

In [12]:
svr_lin.fit(train_X,train_y)
svr_lin.score(test_X,test_y)

0.6083084061232793

In [10]:
svr_poly.fit(train_X,train_y)
svr_poly.score(test_X,test_y)

0.7672391176907403

In [18]:
from sklearn.kernel_ridge import KernelRidge
krr = KernelRidge(alpha=0.1)
krr.fit(train_X,train_y)
krr.score(test_X,test_y)

0.6267532936392413

In [16]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
reg = make_pipeline(StandardScaler(), SGDRegressor())
reg.fit(train_X,train_y)
reg.score(test_X,test_y)

0.6262912813637975